In [2]:
import numpy as np
import cv2
import pickle
import glob
import matplotlib.pyplot as plt
import math
import time
from PIL import Image

import sylvester

#Calibration_Input
import Calibration_Input as CI

CI.Calibration_Input()

# termination criteria
# cv2.TERM_CRITERIA_EPS : 정해둔 오차범위내에 다다르면, 알고리즘을 멈춘다.
# cv2.TERM_CRITERIA_MAX_ITER : 지정한 횟수에 다다르면, 알고리즘을 멈춘다.
#(알고리즘 제동조건, 횟수, 오차범위)

w = int(input("체스판 가로의 꼭지점 수"))
h = int(input("체스판 세로의 꼭지점 수"))

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# object point들을 미리 설정

#objp = np.zeros((체스판 가로, 체스판 세로)) 배열 선언 및 초기화
objp = np.zeros((w*h,3), np.float32)

#objp의 모든행의 [0:2]인덱스 부분에 번호를 부여
objp[:,:2] = np.mgrid[0:h,0:w].T.reshape(-1,2)

# 모든 이미지에서 객체 포인트와 이미지 포인트를 저장하는 배열
# Real World에서의 3d 포인트들을 넣을 배열
objpoints = []

# image plane에서의 2d 포인트들을 넣을 배열
imgpoints = []

images = glob.glob('./calibration_data/*.jpg')
RobotParameters = glob.glob('./calibration_data/*.pickle')
        
for fname in images:
    #사진과 Robot_Matrix가 매칭되는지 검사
    i = fname.replace('jpg', 'xyz')
    
    for k in RobotParameters:
        RM = k.replace('pickle', 'xyz')
        
        if i == RM:
            img = cv2.imread(fname)

            #흑백으로 변경
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # 체스보드 코너들을 찾는 부분
            ret, corners = cv2.findChessboardCorners(gray, (6,4),None)

            # 만약, 찾았으면, objpoints, imgpoints에 추가
            if ret == True:
                objpoints.append(objp)

                #입력 영상 image에서 검출된 코너점 corners를 입력하여 코너점의 위치를 부화소 수준으로 다시 계산하여 반환한다.
                #cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria)
                corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
                imgpoints.append(corners2)

                # 검출한 코너들을 보여주는 부분, 현재, 응답오류로 인해서 제외시켰다.
                img = cv2.drawChessboardCorners(img, (h,w), corners2,ret)

                img = cv2.resize(img, dsize=(640, 480), interpolation=cv2.INTER_AREA)

                #cv2.imshow('img', img)
                #cv2.waitKey(5000)
            else:
                print("검출이 되지 않습니다!")

            cv2.destroyAllWindows()
            

            # 내부 파라미터 : camera calibration 결과, cameramatrix, 왜곡계수, 회전벡터, 병진벡터
            _, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
            _, rvec, tvec, inlier = cv2.solvePnPRansac(objp, corners2, mtx, dist)
            rotM = cv2.Rodrigues(rvec)[0]

            #4x4 Matrix형태로 변환
            A = np.hstack([rotM, tvec])
            A = np.vstack([A, [0, 0, 0, 1]])
            
            RM = RM.replace('xyz', 'pickle')

            with open(RM, "rb") as fr:
                B = pickle.load(fr)
                
            D = sylvester.syl(A, B)
            F = np.zeros((12, 1))
            
            F[3] = -A[0][3]
            F[7] = -A[1][3]
            F[11] = -A[2][3]
            
            X = np.linalg.solve(D, F)
            
            Result = np.zeros((4, 4))

            k = 0

            for i in range(4):
                for j in range(4):
                    if(k <=11):
                        Result[i][j] = X[k]
                        k = k + 1
        
            Result[3][3] = 1
            print(Result)

체스판 가로의 꼭지점 수4
체스판 세로의 꼭지점 수6
[[-0.97191575 -0.15276329 -0.09052527  1.809536  ]
 [-0.12891314  0.53895497 -0.90945125  1.02266406]
 [-0.02377413 -0.04084157  0.12138027  5.30257481]
 [ 0.          0.          0.          1.        ]]
검출이 되지 않습니다!
[[ 20.24459589  -1.78006642   6.68438746 -28.62640567]
 [ -4.5629591  -11.67059715  11.99772766   1.42307252]
 [ -1.95443199   0.14547912  14.56908933 -37.79679875]
 [  0.           0.           0.           1.        ]]
[[-3.52631339e+00 -2.29077444e+00 -4.85885059e+00  1.75243176e+02]
 [ 7.59132556e-01  4.63192905e-01  1.09266611e+00 -4.20044604e+02]
 [ 4.69839139e-01  3.04943454e-01  6.63412316e-01  1.41917316e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-4.67037223e-01 -4.81105052e-02 -5.94381632e-02  4.92104077e+01]
 [ 2.96178831e-02  3.18076734e-03  4.33661873e-03 -2.71394032e+02]
 [ 3.08882308e-02  3.23177563e-03  4.29744061e-03  2.51871017e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000